# Demo af `request`

Et python lib til http-requests

Jeg har nogle filer fra mit løbe ur, med geo-punkter, men højde data er meget uprecise, så jeg vil have noget bedre.
Derfor vil jeg hente fra __opentopodata.org__.

In [5]:
pip install requests


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import requests
res = requests.get('https://api.opentopodata.org/v1/test-dataset?locations=55.73197203068785,12.343950949062537')
print(res.text)


{
  "results": [
    {
      "dataset": "test-dataset",
      "elevation": 32.124603271484375,
      "location": {
        "lat": 55.73197203068785,
        "lng": 12.343950949062537
      }
    }
  ],
  "status": "OK"
}



Det kan jeg godt gøre lidt pænere

In [7]:
import time

# https://api.opentopodata.org/v1/eudem25m?locations=55.732009,12.343829

lat = 55.921015329658985
long = 11.662305062636733

params = {'locations': f"{lat},{long}"}
url  = 'https://api.opentopodata.org/v1/eudem25m'

r = requests.get(url, params=params)

print(r)
print(r.text)

print(r.json()['results'][0]['elevation'])

<Response [200]>
{
  "results": [
    {
      "dataset": "eudem25m",
      "elevation": 14.055695533752441,
      "location": {
        "lat": 55.921015329658985,
        "lng": 11.662305062636733
      }
    }
  ],
  "status": "OK"
}

14.055695533752441


Det var noget med nogle filer...

De er i et propriotært binært format `.FIT`, så det skal indlæses med 

In [8]:
pip install pytz fitdecode


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pytz
from fit_file import read
    
fname = "data/hok_klubmesterskab_2022/CA8D1347.FIT"
points = read.read_points(fname)
print(points)


[{'latitude': 55.921015329658985, 'longitude': 11.662305062636733, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 47, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 98, 'cadence': 0}, {'latitude': 55.92102589085698, 'longitude': 11.662357868626714, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 48, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 97, 'cadence': 61}, {'latitude': 55.92099596746266, 'longitude': 11.662495248019695, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 54, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 92, 'cadence': 73}, {'latitude': 55.920978700742126, 'longitude': 11.662526177242398, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 57, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 98, 'cadence': 71}, {'latitude': 55.92

In [10]:
import json
json_points = json.dumps(points, default=str, indent=2)
print(json_points)

[
  {
    "latitude": 55.921015329658985,
    "longitude": 11.662305062636733,
    "altitude": 12.799999999999955,
    "timestamp": "2022-10-08 13:13:47+02:00",
    "heart_rate": 98,
    "cadence": 0
  },
  {
    "latitude": 55.92102589085698,
    "longitude": 11.662357868626714,
    "altitude": 12.799999999999955,
    "timestamp": "2022-10-08 13:13:48+02:00",
    "heart_rate": 97,
    "cadence": 61
  },
  {
    "latitude": 55.92099596746266,
    "longitude": 11.662495248019695,
    "altitude": 12.799999999999955,
    "timestamp": "2022-10-08 13:13:54+02:00",
    "heart_rate": 92,
    "cadence": 73
  },
  {
    "latitude": 55.920978700742126,
    "longitude": 11.662526177242398,
    "altitude": 12.799999999999955,
    "timestamp": "2022-10-08 13:13:57+02:00",
    "heart_rate": 98,
    "cadence": 71
  },
  {
    "latitude": 55.92097459360957,
    "longitude": 11.662537157535553,
    "altitude": 12.600000000000023,
    "timestamp": "2022-10-08 13:13:58+02:00",
    "heart_rate": 103,
    

In [13]:
import time

for p in points:
    lat = p['latitude']
    long = p['longitude']

    params = {'locations': f"{lat},{long}"}
    url  = 'https://api.opentopodata.org/v1/eudem25m'

    r = requests.get(url, params=params)

    print(r)
    print(r.text)

    print(r.json()['results'][0]['elevation'])

    p['elevation'] = r.json()['results'][0]['elevation']
    time.sleep(1)

<Response [200]>
{
  "results": [
    {
      "dataset": "eudem25m",
      "elevation": 14.055695533752441,
      "location": {
        "lat": 55.921015329658985,
        "lng": 11.662305062636733
      }
    }
  ],
  "status": "OK"
}

14.055695533752441
<Response [200]>
{
  "results": [
    {
      "dataset": "eudem25m",
      "elevation": 14.116682052612305,
      "location": {
        "lat": 55.92102589085698,
        "lng": 11.662357868626714
      }
    }
  ],
  "status": "OK"
}

14.116682052612305
<Response [200]>
{
  "results": [
    {
      "dataset": "eudem25m",
      "elevation": 14.143874168395996,
      "location": {
        "lat": 55.92099596746266,
        "lng": 11.662495248019695
      }
    }
  ],
  "status": "OK"
}

14.143874168395996
<Response [200]>
{
  "results": [
    {
      "dataset": "eudem25m",
      "elevation": 14.13538932800293,
      "location": {
        "lat": 55.920978700742126,
        "lng": 11.662526177242398
      }
    }
  ],
  "status": "OK"
}

14

KeyError: 'results'

In [31]:
n = len(points)
wz = 100 # window size

for w in range(0, n, wz):
    print(w)
    # for i in range(w, w+wz):
    #     print(f"{i},", end=" ")
    
    print(points[w: w+wz])

    # for p in points[w: w+100]:
    #     print(p)

    lat_long_str_list = [f"{p['latitude']},{p['longitude']}" for p in points[w: w + wz]]
    lat_log_str = "|".join(lat_long_str_list)
    print(lat_log_str)

    params = {'locations': lat_log_str}
    url  = 'https://api.opentopodata.org/v1/eudem25m'
    r = requests.get(url, params=params)
    if r.status_code != 200:
        print(r.status_code)
        print(r.text)
    results = r.json()['results']
    print(results)
    
    time.sleep(0.5)


0
[{'latitude': 55.921015329658985, 'longitude': 11.662305062636733, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 47, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 98, 'cadence': 0, 'elevation': 14.055695533752441}, {'latitude': 55.92102589085698, 'longitude': 11.662357868626714, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 48, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 97, 'cadence': 61, 'elevation': 14.116682052612305}, {'latitude': 55.92099596746266, 'longitude': 11.662495248019695, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 54, tzinfo=<DstTzInfo 'Europe/Copenhagen' CEST+2:00:00 DST>), 'heart_rate': 92, 'cadence': 73, 'elevation': 14.143874168395996}, {'latitude': 55.920978700742126, 'longitude': 11.662526177242398, 'altitude': 12.799999999999955, 'timestamp': datetime.datetime(2022, 10, 8, 13, 13, 57, tzinfo=<D